# AICUP_影像資料生成競賽

**各項程式碼有路徑的皆有含路徑說明，請注意資料路徑**

## 環境配置

In [2]:
# 連接雲端
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 將程式碼clone至colab檔案處

! git clone https://github.com/Tingouoww/aicup2024.git

Cloning into 'aicup2024'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 101 (delta 28), reused 89 (delta 19), pack-reused 0
Receiving objects: 100% (101/101), 632.11 KiB | 15.05 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [4]:
# 更改當前工作目錄
import os
os.chdir('/content/aicup2024')

In [5]:
# 安裝所需套件
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 47.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvi

## 資料前處理

- 圖片放大及填充
- label img 與 img 合併
- 分割道路與河流圖
- 分割驗證集與訓練集



### 圖片放大與填充

1. 將原始圖片(428*240)放大至長為512像素，寬依此比例放大
2. 未達512 * 512的部分填充成白色

- label_img與img皆要完成

In [11]:
from PIL import Image, ImageOps

def pad_image(input_path, output_path, target_size=(512, 512), padding_color=(255, 255, 255)):
    """
    input_path: 原始圖片路徑
    output_path: 填充後圖片的保存路徑
    target_size: 填充目標大小(寬, 高)
    padding_color: 填充使用顏色
    """
    img = Image.open(input_path)
    ori_size = img.size  # (width, height)

    # 計算比例並調整圖片大小
    ratio = target_size[0] / ori_size[0]
    new_size = tuple([int(x * ratio) for x in ori_size])
    img_resized = img.resize(new_size, Image.ANTIALIAS)

    # 創建新圖片進行填充
    new_img = Image.new("RGB", target_size, padding_color)
    new_img.paste(img_resized, ((target_size[0] - new_size[0]) // 2,(target_size[1] - new_size[1]) // 2))

    # 填充圖片存檔
    new_img.save(output_path)

"""
路徑說明
ori_path 請填入原始圖片所在資料夾之路徑
pad_path 請填入要放置填充後圖片之路徑
"""

ori_path = '/content/drive/MyDrive/2024_AICUP/original_data/A'
pad_path = '/content/aicup2024/padding/A'

if not os.path.exists(pad_path):
    os.makedirs(pad_path)

for filename in os.listdir(ori_path):

  original_path = os.path.join(ori_path, filename)
  padded_path = os.path.join(pad_path, filename)
  pad_image(original_path, padded_path, target_size=(512, 512))

<ipython-input-11-f67d36e8c0fe>:16: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img_resized = img.resize(new_size, Image.ANTIALIAS)


KeyboardInterrupt: 

In [ ]:
from PIL import Image, ImageOps

def pad_image(input_path, output_path, target_size=(512, 512), padding_color=(255, 255, 255)):
    """
    input_path: 原始圖片路徑
    output_path: 填充後圖片的保存路徑
    target_size: 填充目標大小(寬, 高)
    padding_color: 填充使用顏色
    """
    img = Image.open(input_path)
    ori_size = img.size  # (width, height)

    # 計算比例並調整圖片大小
    ratio = target_size[0] / ori_size[0]
    new_size = tuple([int(x * ratio) for x in ori_size])
    img_resized = img.resize(new_size, Image.ANTIALIAS)

    # 創建新圖片進行填充
    new_img = Image.new("RGB", target_size, padding_color)
    new_img.paste(img_resized, ((target_size[0] - new_size[0]) // 2,(target_size[1] - new_size[1]) // 2))

    # 填充圖片存檔
    new_img.save(output_path)

"""
路徑說明
ori_path 請填入原始圖片所在資料夾之路徑
pad_path 請填入要放置填充後圖片之路徑
"""

ori_path = '/content/drive/MyDrive/2024_AICUP/original_data/B'
pad_path = '/content/aicup2024/padding/B'

if not os.path.exists(pad_path):
    os.makedirs(pad_path)

for filename in os.listdir(ori_path):

  original_path = os.path.join(ori_path, filename)
  padded_path = os.path.join(pad_path, filename)
  pad_image(original_path, padded_path, target_size=(512, 512))

### 訓練資料集之二值圖與真實圖合併

In [ ]:
# 訓練資料集二值圖與真實圖合併
"""
路徑說明
--fold_A label_img_path
--fold_B img_path
--fold_AB after_padding_path
"""

!python datasets/combine_A_and_B.py --fold_A /content/aicup2024/padding/A --fold_B /content/aicup2024/padding/B --fold_AB /content/aicup2024/padding/AB

### 分割河流與道路圖
- 將河流圖與道路圖拆分到不同資料集，以便後續分開訓練

In [ ]:
## 分割河流與道路圖

import os
import shutil

"""
路徑說明
data_dir 資料集所在路徑
road_dir 道路資料集所要存放之路徑
river_dir 河流資料集所要存放之路徑
"""

data_dir = '/content/aicup2024/padding/AB'
road_dir = '/content/aicup2024/datasets/road'
river_dir = '/content/aicup2024/datasets/river'

if not os.path.exists(road_dir):
    os.makedirs(road_dir)

if not os.path.exists(river_dir):
    os.makedirs(river_dir)

for file in os.listdir(data_dir):

    image_dir = os.path.join(data_dir, file)

    """
    file.startswith(word)

    若為訓練集 : word = 'TRA_RO'或'TRA_RI'
    若為公開測試集 : word = 'PUB_RO'或'PUB_RI'
    若為私有測試集 : word = 'PRI_RO'或'PRI_RI'
    """

    if file.startswith('TRA_RO'):
        shutil.copy(image_dir, road_dir)
    elif file.startswith('TRA_RI'):
        shutil.copy(image_dir, river_dir)

### 分割訓練集與驗證集

In [ ]:
# 分割訓練集與驗證集

import os
import random
import shutil

random.seed(20)

"""
路徑說明
data_dir 資料集所在路徑
train_dir 訓練資料集要存放的路徑
val_dir 驗證資料集要存放之路徑
"""

data_dir = '/content/aicup2024/datasets/road'

#訓練資料集比例
train_size = 0.8

file_name = os.listdir(data_dir)
random.shuffle(file_name)
split = int(len(file_name) * train_size)

train_file = file_name[:split]
val_file = file_name[split:]

train_dir = '/content/aicup2024/datasets/road/train'
val_dir = '/content/aicup2024/datasets/road/val'

if not os.path.exists(train_dir):
    os.makedirs(train_dir)

if not os.path.exists(val_dir):
    os.makedirs(val_dir)

for name in train_file:
    source = os.path.join(data_dir, name)
    destination = os.path.join(train_dir, name)
    shutil.move(source, destination)

for name in val_file:
    source = os.path.join(data_dir, name)
    destination = os.path.join(val_dir, name)
    shutil.move(source, destination)

## 檔案複製
- 將已前處理完成的圖片複製到/aicup2024/datasets

In [6]:
## 複製檔案
# road

import os
import shutil

# 原始資料夾
source_path = '/content/drive/MyDrive/2024_AICUP/datasets/road'

# 目標資料夾
target_path = '/content/aicup2024/datasets'

if not os.path.exists(target_path):
    os.makedirs(target_path)

# 複製位置: target_path/file_name
file_name = 'road'
shutil.copytree(source_path, os.path.join(target_path, file_name))

'/content/aicup2024/datasets/road'

In [5]:
## 複製檔案
# river

import os
import shutil

# 原始資料夾
source_path = '/content/drive/MyDrive/2024_AICUP/datasets/river'

# 目標資料夾
target_path = '/content/aicup2024/datasets'

if not os.path.exists(target_path):
    os.makedirs(target_path)

# 複製位置: target_path/file_name
file_name = 'river'
shutil.copytree(source_path, os.path.join(target_path, file_name))

'/content/aicup2024/datasets/river'

## 模型訓練

### river模型訓練

1. river_v9 以lr = 0.0001, lambda_L1 = 110 進行訓練，最終取用100_net.pth進行第二輪訓練
2. river_v9_fine_v1 以lr = 0.0001, lambda_L1 = 110, river_v9->100_net.pth進行訓練, 最終取用30_net.pth進行下一輪訓練
3. river_v9_fine_v2 以lr = 0.0001進行線性衰減， lambda_L1 = 100, river_v9_fine_v1->30_net.pth進行訓練，最終採用90_net.pth進行測試

- 其餘參數參照程式碼設定

In [7]:
# 運行至100epoch結束後手動停止

! python train.py --dataroot ./datasets/river --name river_v9 --model pix2pix --direction AtoB --preprocess none --load_size 512 --norm batch --lr 0.0001 --netG unet_512 --lambda_L1 110.0

----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 256                           
                 dataroot: ./datasets/river              	[default: None]
             dataset_mode: aligned                       
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: 1                             
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
                    epoch: latest                        
              

In [ ]:
# 將river_v9的100_net_G.pth與100_net_D.pth(或是停在100epoch用latest_net)放入/checkpoints/river_v9_fine_v1
# 使用上面river_v9的100_net.pth繼續訓練
# 運行至30epoch結束後手動停止

! python train.py --dataroot ./datasets/river --name river_v9_fine_v1 --model pix2pix --direction AtoB --preprocess none --load_size 512 --norm batch --lr 0.0001 --netG unet_512 --lambda_L1 110.0 --continue_train --epoch 100 --epoch_count 1

In [ ]:
# 將river_v9的30_net_G.pth與30_net_D.pth(或是停在30epoch用latest_net)放入/checkpoints/river_v9_fine_v2
# 使用上面river_v9_fine_v1的30_net.pth繼續訓練
# 運行至90epoch結束後手動停止

! python train.py --dataroot ./datasets/river --name river_v9_fine_v2 --model pix2pix --direction AtoB --preprocess none --load_size 512 --norm batch --lr 0.0001 --netG unet_512 --lambda_L1 100.0 --continue_train --epoch_count 1 --n_epochs 0 --n_epochs_decay 100

### road模型訓練

1. lambda_L1 = 115 訓練100個epoch
2. 將lambda_L1調整成110，使用100_net.pth從epoch = 101開始再訓練，最終採用170_net.pth進行測試

- 其餘參數設定如程式碼

In [ ]:
# road_v12模型訓練

# 運行至100epoch結束後手動停止

! python train.py --dataroot ./datasets/road --name road_v12 --model pix2pix --direction AtoB --preprocess none --load_size 512 --norm batch --lr 0.00015 --netG unet_512 --lambda_L1 115.0 --n_epochs 50 --n_epochs_decay 150

In [ ]:
# 運行至170epoch結束後手動停止

! python train.py --dataroot ./datasets/road --name road_v12 --model pix2pix --direction AtoB --preprocess none --load_size 512 --norm batch --lr 0.00015 --netG unet_512 --lambda_L1 110.0 --n_epochs 50 --n_epochs_decay 150 --continue_train --epoch_count 101 --epoch 100

## 模型驗證

### 河流模型驗證

In [ ]:
! python test.py --dataroot ./datasets/river --name river_v9_fine_v2 --model pix2pix --direction AtoB --norm batch --load_size 512 --preprocess none --results_dir ./results/val_river_v9_fine_v2 --phase val --netG unet_512 --epoch 90

### 道路模型驗證

In [ ]:
! python test.py --dataroot ./datasets/road --name road_v12 --model pix2pix --direction AtoB --norm batch --load_size 512 --preprocess none --results_dir ./results/val_road_v12 --phase val --netG unet_512 --epoch 170

## 生成圖片

### 河流圖片生成

In [ ]:
# public testing datasets

! python test.py --dataroot ./datasets/river --name river_v9_fine_v2 --model test --direction AtoB --norm batch --load_size 512 --preprocess none --results_dir ./results/pub_river_v9_fine_v2 --phase test --netG unet_512 --epoch 90

In [ ]:
# private testing datasets

! python test.py --dataroot ./datasets/pri_river --name river_v9_fine_v2 --model test --direction AtoB --norm batch --load_size 512 --preprocess none --results_dir ./results/pri_river_v9_fine_v2 --phase test --netG unet_512 --epoch 90

### 道路圖片生成

In [ ]:
# public testing datasets

! python test.py --dataroot ./datasets/road --name road_v12 --model test --netG unet_512 --direction AtoB --norm batch --load_size 512 --preprocess none --results_dir ./results/pub_road_v12 --epoch 170 --num_test 360

In [ ]:
# private testing datasets

! python test.py --dataroot ./datasets/pri_road --name road_v12 --model test --netG unet_512 --direction AtoB --norm batch --load_size 512 --preprocess none --results_dir ./results/pri_road_v12 --epoch 170 --num_test 360

## 生成圖片還原大小

- 由於訓練時做了前處理，改變了圖片大小，因此生成完後進行相對應還原處理

In [ ]:
"""
競賽繳交規定為.jpg檔案，因此進行轉檔
"""

import os
import shutil
from PIL import Image

"""
路徑說明
source_folder 要轉檔資料的所在路徑
destination_folder 轉檔後存放的路徑
"""
# 定義來源資料夾和目標資料夾的路徑
source_folder = '/content/drive/MyDrive/dataset_GAN/A_padding/river'
destination_folder = '/content/drive/MyDrive/dataset_GAN/result/pri_river_v14_new'

if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

for filename in os.listdir(source_folder):

    """
    以下filename.endswith('_fake.png') and filename.startswith('PRI')
    在針對公開測試集時，請改成filename.endswith('_fake.png') and filename.startswith('PUB')
    """

    if filename.endswith('_fake.png') and filename.startswith('PRI'):
        source_path = os.path.join(source_folder, filename)
        destination_path = os.path.join(destination_folder, filename)

        img = Image.open(source_path)
        new_filename = filename[:-9] + '.jpg'
        new_destination_path = os.path.join(destination_folder, new_filename)
        img.convert('RGB').save(new_destination_path, 'JPEG')

In [ ]:
"""
進行去除填充部分與還原大小
運行完成後得生成圖片最終結果
"""
from PIL import Image
import os

def crop_image_to_original_size(padded_image_path, output_path, original_size, target_size=(512, 512)):
    """
    從填充後的圖片中裁剪出原始大小的圖片
    padded_image_path: 填充後圖片的路徑
    output_path: 裁剪後圖片的保存路徑
    original_size: 原始圖片的大小 (寬, 高)
    target_size: 填充後的目標尺寸 (寬, 高)
    """
    # 讀取填充後的圖片
    padded_img = Image.open(padded_image_path)

    # 計算比例並調整圖片大小
    ratio = min(target_size[0] / original_size[0], target_size[1] / original_size[1])
    new_size = (int(original_size[0] * ratio), int(original_size[1] * ratio))

    # 計算填充的起始點
    left = (target_size[0] - new_size[0]) // 2
    top = (target_size[1] - new_size[1]) // 2
    right = left + new_size[0]
    bottom = top + new_size[1]

    # 剪裁圖片
    original_img = padded_img.crop((left, top, right, bottom))

    # 按比例縮放回原始尺寸
    original_img_resized = original_img.resize(original_size, Image.ANTIALIAS)

    # 保存裁剪並調整尺寸後的圖片
    original_img_resized.save(output_path)

"""
路徑說明
padded_folder 存放要還原的填充圖片路徑
output_folder 最終繳交檔案存放路徑
"""

original_size = (428, 240)  # 規定圖片大小
padded_folder = ''
output_folder = ''

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for filename in os.listdir(padded_folder):
    padded_image_path = os.path.join(padded_folder, filename)
    output_image_path = os.path.join(output_folder, filename)
    crop_image_to_original_size(padded_image_path, output_image_path, original_size)